# The-Numbers.com Box office Analysis

## Import Packages

In [370]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from datetime import datetime
import copy
import holidays
import pickle

In [371]:
path = 'raw_data'
files = os.listdir(path)

## Loading Data

In [372]:
# Loading the data
df_details = pd.DataFrame()
df_bo = pd.DataFrame()
df_int = pd.DataFrame()
df_country = pd.DataFrame()

for file in files:
    if re.match('(\d+)_movie_details\.csv', file):
        df_details = pd.concat([df_details, pd.read_csv(path + '/' + file)])
    if re.match('(\d+)_bo_performance\.csv', file):
        df_bo = pd.concat([df_bo, pd.read_csv(path + '/' + file)])
    if re.match('(\d+)_international_bo_performance\.csv', file):
        df_int = pd.concat([df_int, pd.read_csv(path + '/' + file)])
    if re.match('(\d+)_country_bo_performance\.csv', file):
        df_country = pd.concat([df_country, pd.read_csv(path + '/' + file)])

df_movies = pd.read_csv(path + '/' +'20230724_movie_list.csv')

/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/3711477783.py:15: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_country = pd.concat([df_country, pd.read_csv(path + '/' + file)])
/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/3711477783.py:15: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_country = pd.concat([df_country, pd.read_csv(path + '/' + file)])


In [373]:
df_details

,Unnamed: 0,url,cast,crew,Opening Weekend:,Legs:,Domestic Share:,Theater counts:,Infl. Adj. Dom. BO,Production Budget:,...,Production/Financing Companies:,Production Countries:,Languages:,Franchise:,Domestic Box Office,International Box Office,Worldwide Box Office,Est. Domestic DVD Sales,Est. Domestic Blu-ray Sales,Total Est. Domestic Video Sales
0,0,https://www.the-numbers.com/movie/Dry-The-(Aus...,"[""Genevieve O'Reilly"", ""Keir O'Donnell"", 'John...","[['Robert Connolly', 'Director'], ['Harry Crip...","$119,364 (32.8% of total gross)",3.05 (domestic box office/biggest weekend),2.1% (domestic box office/worldwide),"186 opening theaters/186 max. theaters, 2.5 we...","$366,147",NaN,...,"Screen Australia, Film Victoria, Made Up Stori...","Australia, United States",English,NaN,"$364,397","$17,124,326","$17,488,723",NaN,NaN,NaN
1,1,https://www.the-numbers.com/movie/Octonauts-Th...,[],"[['Blair Simmons', 'Director'], ['Qin Zhou', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"China, United Kingdom",Mandarin,NaN,NaN,"$8,437,314","$8,437,314",NaN,NaN,NaN
2,2,https://www.the-numbers.com/movie/Penguin-Bloom,"['Andrew Lincoln', 'Jacki Weaver', 'Rachel Hou...","[['Glendyn Ivin', 'Director'], ['Shaun Grant',...",NaN,NaN,NaN,NaN,NaN,NaN,...,"Made Up Stories, Screen Australia, Broadtalk, ...","Australia, United States",English,NaN,NaN,"$6,262,056","$6,262,056",NaN,NaN,NaN
3,3,https://www.the-numbers.com/movie/Posledniy-bo...,[],"[['Dmitry Dyachenko', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Russian Federation,Russian,NaN,NaN,"$33,396,899","$33,396,899",NaN,NaN,NaN
4,4,https://www.the-numbers.com/movie/Gintama-The-...,[],"[['Chizuru Miyawaki', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Japan,Japanese,Gintama,NaN,"$15,775,367","$15,775,367",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,111,https://www.the-numbers.com/movie/Kindergarten...,"['Penelope Ann Miller', 'Linda Hunt', 'Richard...","[['Ivan Reitman', 'Director'], ['Ivan Reitman'...","$11,260,685 (12.3% of total gross)",5.59 (domestic box office/biggest weekend),45.3% (domestic box office/worldwide),"1,833 opening theaters/1,934 max. theaters, 9....","$198,963,691","$26,000,000 (worldwide box office is 7.8 times...",...,Imagine Entertainment,United States,English,Kindergarten Cop,"$91,457,688","$110,542,312","$202,000,000",NaN,NaN,NaN
112,112,https://www.the-numbers.com/movie/Long-Walk-Ho...,[],"[['Richard Pearce', 'Director'], ['John Cork',...",NaN,20.43 (domestic box office/biggest weekend),100.0% (domestic box office/worldwide),NaN,"$30,054,134",NaN,...,NaN,United States,NaN,NaN,"$13,798,030",NaN,NaN,NaN,NaN,NaN
113,113,https://www.the-numbers.com/movie/Alice-(1990),[],"[['Woody Allen', 'Director'], ['Woody Allen', ...",NaN,NaN,100.0% (domestic box office/worldwide),NaN,"$15,931,564",NaN,...,NaN,United States,"Cantonese, English",NaN,"$7,331,647",NaN,NaN,NaN,NaN,NaN
114,114,https://www.the-numbers.com/movie/Godfather-Pa...,"['Andy Garcia', 'Al Pacino']","[['Francis Ford Coppola', 'Director'], ['Mario...","$19,558,558 (29.3% of total gross)",3.41 (domestic box office/biggest weekend),99.8% (domestic box office/worldwide),"1,901 opening theaters/1,922 max. theaters, 6....","$144,909,420","$54,000,000 (worldwide box office is 1.2 times...",...,Paramount Pictures,United States,"English, German, Italian, Latin",Godfather,"$66,666,062","$120,904","$66,786,966",NaN,NaN,NaN


## Core Functions

### Cleaning Monetary values

In [374]:
# Cleaning Columns that contain money values
def get_monetary_clean(df, cols):
    for col in cols:
        df[col] = (pd.to_numeric(df[col]
                                 .str.replace(',', '', regex=False)
                                 .str.replace('$', '', regex=False)
                                 .str.replace('\xa0', '', regex=False), errors='coerce', downcast='float'))

### Setting as categorical

In [375]:
def set_categorical(df, vars):
    df[vars] = df[vars].astype('category')

### Filter in list

In [376]:
def filter_list_col(df, col, search_string):
    df = (df.loc[df[col].apply(lambda x: search_string in x if isinstance(x, list) else False), :])
    return df

### Format as Date

In [377]:
def set_as_date(df, col, date_format):
    df[col] = df[col].apply(lambda x: datetime.strptime(x, date_format).date() if x != 'nan' else None).astype('datetime64')

### Crew Top Roles

In [378]:
def crew_top_roles(start, end):
    _comb_dict = {} 

    for dictionary in movies['crew']:
        copy_dicts = copy.deepcopy(dictionary[0])
        for orig_key, orig_value in copy_dicts.items():
            key = orig_key
            value = orig_value
            if key in _comb_dict:
                _comb_dict[key].append(value)
            else:
                _comb_dict[key] = value

    value_counts = {key: len(values) for key, values in _comb_dict.items()}
    _top_roles = pd.Series(value_counts)
    print(len(_top_roles))

    print(_top_roles.sort_values(ascending=False)[start:end])

### Format Numbers

In [379]:
def numformat(df, columns):
    return df[columns].applymap(lambda x: f'{x/1000000:.3f}M' if isinstance(x, float) else x)

# Data Cleaning

In [380]:
df_movies.head()

,Unnamed: 0,year,movie,url,genre,release_type,revenue_to_date
0,23,1915,The Birth of a Nation,https://www.the-numbers.com/movie/Birth-of-a-N...,Drama,Theatrical,11000000.0
1,41,1916,"20,000 Leagues Under the Sea",https://www.the-numbers.com/movie/20000-League...,Adventure,Theatrical,8000000.0
2,77,1921,The Four Horsemen of the Apocalypse,https://www.the-numbers.com/movie/Four-Horseme...,NaN,Theatrical,9200000.0
3,148,1925,The Big Parade,https://www.the-numbers.com/movie/Big-Parade-T...,Drama,Theatrical,22268719.0
4,155,1925,Ben-Hur: A Tale of the Christ,https://www.the-numbers.com/movie/Ben-Hur-A-Ta...,Adventure,Theatrical,9000000.0


In [381]:
df_movies = df_movies.drop('Unnamed: 0', axis=1)

In [382]:
df_details.head()

,Unnamed: 0,url,cast,crew,Opening Weekend:,Legs:,Domestic Share:,Theater counts:,Infl. Adj. Dom. BO,Production Budget:,...,Production/Financing Companies:,Production Countries:,Languages:,Franchise:,Domestic Box Office,International Box Office,Worldwide Box Office,Est. Domestic DVD Sales,Est. Domestic Blu-ray Sales,Total Est. Domestic Video Sales
0,0,https://www.the-numbers.com/movie/Dry-The-(Aus...,"[""Genevieve O'Reilly"", ""Keir O'Donnell"", 'John...","[['Robert Connolly', 'Director'], ['Harry Crip...","$119,364 (32.8% of total gross)",3.05 (domestic box office/biggest weekend),2.1% (domestic box office/worldwide),"186 opening theaters/186 max. theaters, 2.5 we...","$366,147",NaN,...,"Screen Australia, Film Victoria, Made Up Stori...","Australia, United States",English,NaN,"$364,397","$17,124,326","$17,488,723",NaN,NaN,NaN
1,1,https://www.the-numbers.com/movie/Octonauts-Th...,[],"[['Blair Simmons', 'Director'], ['Qin Zhou', '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"China, United Kingdom",Mandarin,NaN,NaN,"$8,437,314","$8,437,314",NaN,NaN,NaN
2,2,https://www.the-numbers.com/movie/Penguin-Bloom,"['Andrew Lincoln', 'Jacki Weaver', 'Rachel Hou...","[['Glendyn Ivin', 'Director'], ['Shaun Grant',...",NaN,NaN,NaN,NaN,NaN,NaN,...,"Made Up Stories, Screen Australia, Broadtalk, ...","Australia, United States",English,NaN,NaN,"$6,262,056","$6,262,056",NaN,NaN,NaN
3,3,https://www.the-numbers.com/movie/Posledniy-bo...,[],"[['Dmitry Dyachenko', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Russian Federation,Russian,NaN,NaN,"$33,396,899","$33,396,899",NaN,NaN,NaN
4,4,https://www.the-numbers.com/movie/Gintama-The-...,[],"[['Chizuru Miyawaki', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Japan,Japanese,Gintama,NaN,"$15,775,367","$15,775,367",NaN,NaN,NaN


In [383]:
df_details = df_details.drop(['Unnamed: 0'], axis=1)
df_details.head()

,url,cast,crew,Opening Weekend:,Legs:,Domestic Share:,Theater counts:,Infl. Adj. Dom. BO,Production Budget:,Domestic Releases:,...,Production/Financing Companies:,Production Countries:,Languages:,Franchise:,Domestic Box Office,International Box Office,Worldwide Box Office,Est. Domestic DVD Sales,Est. Domestic Blu-ray Sales,Total Est. Domestic Video Sales
0,https://www.the-numbers.com/movie/Dry-The-(Aus...,"[""Genevieve O'Reilly"", ""Keir O'Donnell"", 'John...","[['Robert Connolly', 'Director'], ['Harry Crip...","$119,364 (32.8% of total gross)",3.05 (domestic box office/biggest weekend),2.1% (domestic box office/worldwide),"186 opening theaters/186 max. theaters, 2.5 we...","$366,147",NaN,"May 21st, 2021 (Limited) by IFC Films",...,"Screen Australia, Film Victoria, Made Up Stori...","Australia, United States",English,NaN,"$364,397","$17,124,326","$17,488,723",NaN,NaN,NaN
1,https://www.the-numbers.com/movie/Octonauts-Th...,[],"[['Blair Simmons', 'Director'], ['Qin Zhou', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"China, United Kingdom",Mandarin,NaN,NaN,"$8,437,314","$8,437,314",NaN,NaN,NaN
2,https://www.the-numbers.com/movie/Penguin-Bloom,"['Andrew Lincoln', 'Jacki Weaver', 'Rachel Hou...","[['Glendyn Ivin', 'Director'], ['Shaun Grant',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Made Up Stories, Screen Australia, Broadtalk, ...","Australia, United States",English,NaN,NaN,"$6,262,056","$6,262,056",NaN,NaN,NaN
3,https://www.the-numbers.com/movie/Posledniy-bo...,[],"[['Dmitry Dyachenko', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Russian Federation,Russian,NaN,NaN,"$33,396,899","$33,396,899",NaN,NaN,NaN
4,https://www.the-numbers.com/movie/Gintama-The-...,[],"[['Chizuru Miyawaki', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Japan,Japanese,Gintama,NaN,"$15,775,367","$15,775,367",NaN,NaN,NaN


In [384]:
#there seem to be duplicate values 
display(df_details.duplicated().sum())
display(df_movies.duplicated().sum())

2

0

### Handling movies duplicate values based on url

In [385]:
# Dropping duplicates
df_details = df_details.drop_duplicates()

In [386]:
# Checking if the shape of the two match up
display(df_details.shape)
display(df_movies.shape)


(9526, 30)

(9528, 6)

In [387]:
movies = df_details.merge(df_movies, on=['url'])
movies.head()

,url,cast,crew,Opening Weekend:,Legs:,Domestic Share:,Theater counts:,Infl. Adj. Dom. BO,Production Budget:,Domestic Releases:,...,International Box Office,Worldwide Box Office,Est. Domestic DVD Sales,Est. Domestic Blu-ray Sales,Total Est. Domestic Video Sales,year,movie,genre,release_type,revenue_to_date
0,https://www.the-numbers.com/movie/Dry-The-(Aus...,"[""Genevieve O'Reilly"", ""Keir O'Donnell"", 'John...","[['Robert Connolly', 'Director'], ['Harry Crip...","$119,364 (32.8% of total gross)",3.05 (domestic box office/biggest weekend),2.1% (domestic box office/worldwide),"186 opening theaters/186 max. theaters, 2.5 we...","$366,147",NaN,"May 21st, 2021 (Limited) by IFC Films",...,"$17,124,326","$17,488,723",NaN,NaN,NaN,2021,The Dry,Thriller/Suspense,Theatrical,17488723.0
1,https://www.the-numbers.com/movie/Octonauts-Th...,[],"[['Blair Simmons', 'Director'], ['Qin Zhou', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$8,437,314","$8,437,314",NaN,NaN,NaN,2021,Octonauts: The Ring of Fire,Adventure,Theatrical,8437314.0
2,https://www.the-numbers.com/movie/Penguin-Bloom,"['Andrew Lincoln', 'Jacki Weaver', 'Rachel Hou...","[['Glendyn Ivin', 'Director'], ['Shaun Grant',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$6,262,056","$6,262,056",NaN,NaN,NaN,2021,Penguin Bloom,Drama,Theatrical,6262056.0
3,https://www.the-numbers.com/movie/Posledniy-bo...,[],"[['Dmitry Dyachenko', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$33,396,899","$33,396,899",NaN,NaN,NaN,2021,Posledniy bogatyr. Koren’ Zla,Adventure,Theatrical,33396899.0
4,https://www.the-numbers.com/movie/Gintama-The-...,[],"[['Chizuru Miyawaki', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$15,775,367","$15,775,367",NaN,NaN,NaN,2021,Gintama: The Final,Action,Theatrical,15775367.0


In [388]:
_duplicated_id = movies.loc[movies['url'].duplicated(keep=False), :]

In [389]:
# investigating where differences are present
_duplicated_id.nunique().sort_values(ascending=False).head()

revenue_to_date                    4
Keywords:                          2
movie                              2
year                               2
Total Est. Domestic Video Sales    2
dtype: int64

In [390]:
_duplicated_id['revenue_to_date']

2011    214286622.0
2012    214287001.0
2798    244154308.0
2799    244153251.0
Name: revenue_to_date, dtype: float64

In [391]:
#Minimal differences, just go ahead and drop
movies.drop_duplicates(subset='url', inplace=True)

### Cleaning columns

In [392]:
# investigating Columns
movies.columns

Index(['url', 'cast', 'crew', 'Opening Weekend:', 'Legs:', 'Domestic Share:',
       'Theater counts:', 'Infl. Adj. Dom. BO', 'Production Budget:',
       'Domestic Releases:', 'International Releases:', 'Video Release:',
       'MPAA Rating:', 'Running Time:', 'Comparisons:', 'Keywords:', 'Source:',
       'Genre:', 'Production Method:', 'Creative Type:',
       'Production/Financing Companies:', 'Production Countries:',
       'Languages:', 'Franchise:', 'Domestic Box Office',
       'International Box Office', 'Worldwide Box Office',
       'Est. Domestic DVD Sales', 'Est. Domestic Blu-ray Sales',
       'Total Est. Domestic Video Sales', 'year', 'movie', 'genre',
       'release_type', 'revenue_to_date'],
      dtype='object')

In [393]:
movies.rename(inplace=True, columns={
    'Domestic Share:': 'dom_pct',
    'Production\xa0Budget:': 'budget',
    'Legs:': 'legs',
    'Opening\xa0Weekend:': 'opening_wknd_bo',
    'Theater counts:': 'n_theaters',
    'Infl. Adj. Dom. BO': 'adj_dom_bo',
    'Domestic Releases:': 'dom_releases',
    'International Releases:': 'int_releases',
    'Video\xa0Release:': 'video_releases',
    'MPAA\xa0Rating:': 'mpaa',
    'Running Time:': 'runtime',
    'Comparisons:': 'comps',
    'Keywords:': 'keywords',
    'Source:': 'source',
    'Genre:': 'genre_details',
    'Production\xa0Method:': 'prod_method',
    'Creative\xa0Type:': 'creative_type',
    'Production Countries:': 'prod_countries',
    'Languages:': 'languages',
    'Franchise:': 'franchise',
    'Production/Financing Companies:': 'producers',
    'Domestic Box Office': 'dom_bo',
    'International Box Office': 'int_bo',
    'Worldwide Box Office': 'ww_bo',
    'Est. Domestic Blu-ray Sales': 'est_dom_bluray',
    'Total Est. Domestic Video Sales': 'tot_dom_vid_sales',
    'Est. Domestic DVD Sales': 'est_dom_dvd',
    'movie': 'movie_title'
})

In [394]:
movies.columns

Index(['url', 'cast', 'crew', 'opening_wknd_bo', 'legs', 'dom_pct',
       'n_theaters', 'adj_dom_bo', 'budget', 'dom_releases', 'int_releases',
       'video_releases', 'mpaa', 'runtime', 'comps', 'keywords', 'source',
       'genre_details', 'prod_method', 'creative_type', 'producers',
       'prod_countries', 'languages', 'franchise', 'dom_bo', 'int_bo', 'ww_bo',
       'est_dom_dvd', 'est_dom_bluray', 'tot_dom_vid_sales', 'year',
       'movie_title', 'genre', 'release_type', 'revenue_to_date'],
      dtype='object')

In [395]:
movies.head()

,url,cast,crew,opening_wknd_bo,legs,dom_pct,n_theaters,adj_dom_bo,budget,dom_releases,...,int_bo,ww_bo,est_dom_dvd,est_dom_bluray,tot_dom_vid_sales,year,movie_title,genre,release_type,revenue_to_date
0,https://www.the-numbers.com/movie/Dry-The-(Aus...,"[""Genevieve O'Reilly"", ""Keir O'Donnell"", 'John...","[['Robert Connolly', 'Director'], ['Harry Crip...","$119,364 (32.8% of total gross)",3.05 (domestic box office/biggest weekend),2.1% (domestic box office/worldwide),"186 opening theaters/186 max. theaters, 2.5 we...","$366,147",NaN,"May 21st, 2021 (Limited) by IFC Films",...,"$17,124,326","$17,488,723",NaN,NaN,NaN,2021,The Dry,Thriller/Suspense,Theatrical,17488723.0
1,https://www.the-numbers.com/movie/Octonauts-Th...,[],"[['Blair Simmons', 'Director'], ['Qin Zhou', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$8,437,314","$8,437,314",NaN,NaN,NaN,2021,Octonauts: The Ring of Fire,Adventure,Theatrical,8437314.0
2,https://www.the-numbers.com/movie/Penguin-Bloom,"['Andrew Lincoln', 'Jacki Weaver', 'Rachel Hou...","[['Glendyn Ivin', 'Director'], ['Shaun Grant',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$6,262,056","$6,262,056",NaN,NaN,NaN,2021,Penguin Bloom,Drama,Theatrical,6262056.0
3,https://www.the-numbers.com/movie/Posledniy-bo...,[],"[['Dmitry Dyachenko', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$33,396,899","$33,396,899",NaN,NaN,NaN,2021,Posledniy bogatyr. Koren’ Zla,Adventure,Theatrical,33396899.0
4,https://www.the-numbers.com/movie/Gintama-The-...,[],"[['Chizuru Miyawaki', 'Director']]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"$15,775,367","$15,775,367",NaN,NaN,NaN,2021,Gintama: The Final,Action,Theatrical,15775367.0


### Cleaning `release_type`

In [396]:
# There is only one release type, hence we can drop it
movies['release_type'].unique()

array(['Theatrical'], dtype=object)

In [397]:
movies = movies.drop(columns='release_type')

### Cleaning `legs`

In [398]:
# Checking out legs
movies.loc[~movies['legs'].isna(), 'legs'].head()

0     3.05 (domestic box office/biggest weekend)
8     5.01 (domestic box office/biggest weekend)
13    9.97 (domestic box office/biggest weekend)
14    3.26 (domestic box office/biggest weekend)
16    5.17 (domestic box office/biggest weekend)
Name: legs, dtype: object

In [399]:
# Seems like the latter part does not add any value. lets remove it 
movies['legs'] = (movies['legs']
                  .str.replace(' (domestic box office/biggest weekend)', '', regex=False)
                  .str.replace(',','', regex=False)
                  .astype('float'))
movies['legs'].head()

0    3.05
1     NaN
2     NaN
3     NaN
4     NaN
Name: legs, dtype: float64

### Cleaning `creative_type`

In [400]:
# Creative type seems fine
movies['creative_type'].unique()

array(['Contemporary Fiction', 'Kids Fiction', 'Dramatization',
       'Super Hero', 'Fantasy', 'Science Fiction', nan,
       'Historical Fiction', 'Multiple Creative Types', 'Factual'],
      dtype=object)

In [401]:
set_categorical(movies, ['creative_type'])
movies['creative_type'].head()

0    Contemporary Fiction
1            Kids Fiction
2           Dramatization
3              Super Hero
4                 Fantasy
Name: creative_type, dtype: category
Categories (9, object): ['Contemporary Fiction', 'Dramatization', 'Factual', 'Fantasy', ..., 'Kids Fiction', 'Multiple Creative Types', 'Science Fiction', 'Super Hero']

### Cleaning `runtime`

In [402]:
movies['runtime'].unique()

array(['118 minutes', nan, '95 minutes', '104 minutes', '125 minutes',
       '130 minutes', '179 minutes', '107 minutes', '90 minutes',
       '98 minutes', '122 minutes', '136 minutes', '109 minutes',
       '127 minutes', '124 minutes', '108 minutes', '110 minutes',
       '126 minutes', '129 minutes', '100 minutes', '111 minutes',
       '75 minutes', '114 minutes', '112 minutes', '87 minutes',
       '154 minutes', '115 minutes', '84 minutes', '119 minutes',
       '92 minutes', '113 minutes', '93 minutes', '91 minutes',
       '96 minutes', '99 minutes', '138 minutes', '120 minutes',
       '83 minutes', '145 minutes', '105 minutes', '102 minutes',
       '134 minutes', '137 minutes', '133 minutes', '94 minutes',
       '97 minutes', '88 minutes', '103 minutes', '131 minutes',
       '121 minutes', '132 minutes', '117 minutes', '89 minutes',
       '101 minutes', '60 minutes', '116 minutes', '156 minutes',
       '176 minutes', '163 minutes', '86 minutes', '123 minutes',
       '

In [403]:
movies['runtime'] = (movies['runtime']
                     .str.replace(' minutes', '')
                     .astype('float'))
movies['runtime'].head()

0    118.0
1      NaN
2     95.0
3      NaN
4    104.0
Name: runtime, dtype: float64

### Cleaning `sources`

In [404]:
#Checking for source
movies['source'].unique()

array(['Based on Fiction Book/Short Story', 'Based on TV',
       'Based on Factual Book/Article', nan,
       'Based on Comic/Graphic Novel', 'Original Screenplay', 'Remake',
       'Compilation', 'Based on Real Life Events', 'Based on Short Film',
       'Based on Folk Tale/Legend/Fairytale', 'Spin-Off', 'Based on Game',
       'Based on Movie', 'Based on Musical or Opera', 'Based on Toy',
       'Based on Theme Park Ride', 'Based on Play',
       'Based on Religious Text', 'Based on Song', 'Based on Ballet',
       'Based on Musical Group', 'Based on Web Series', 'Based on Poem'],
      dtype=object)

In [405]:
# Data seems clean 
set_categorical(movies, ['source'])
movies['source']

0       Based on Fiction Book/Short Story
1                             Based on TV
2           Based on Factual Book/Article
3                                     NaN
4            Based on Comic/Graphic Novel
                      ...                
9523                  Original Screenplay
9524                  Based on Short Film
9525                  Original Screenplay
9526    Based on Fiction Book/Short Story
9527                  Original Screenplay
Name: source, Length: 9526, dtype: category
Categories (23, object): ['Based on Ballet', 'Based on Comic/Graphic Novel', 'Based on Factual Book/Article', 'Based on Fiction Book/Short Story', ..., 'Compilation', 'Original Screenplay', 'Remake', 'Spin-Off']

### Cleaning `genre`

In [406]:
# There seem to be two genre columns. Checking if genre_details == genre
_genre_check = movies['genre_details'] == movies['genre']
movies.loc[~_genre_check, 'genre'].unique()

array([nan], dtype=object)

In [407]:
#Seems that they are identical, hence we can drop one of them
movies = movies.drop(columns='genre_details')

In [408]:
movies['genre'].unique()

array(['Thriller/Suspense', 'Adventure', 'Drama', 'Action',
       'Romantic Comedy', 'Horror', nan, 'Comedy', 'Musical',
       'Black Comedy', 'Documentary', 'Multiple Genres',
       'Concert/Performance', 'Western', 'Reality', 'Educational'],
      dtype=object)

In [409]:
# Looks clean, set categorical
set_categorical(movies, ['genre'])

### Cleaning `budget`

In [410]:
#Checking budget
movies['budget'].unique()

array([nan,
       '$23,000,000 (worldwide box office is 1.0 times production budget)',
       '$117,000,000 (worldwide box office is 6.0 times production budget)',
       ...,
       '$29,000,000 (worldwide box office is 1.8 times production budget)',
       '$26,000,000 (worldwide box office is 7.8 times production budget)',
       '$54,000,000 (worldwide box office is 1.2 times production budget)'],
      dtype=object)

In [411]:
#There seems to be additional relevant information. splitting into budget and prod_multiple
movies[['budget', 'prod_multiple']] = (movies['budget']
                                       .str.split(' ', 1, expand=True))
movies[['budget', 'prod_multiple']].head(10)

/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/2479258636.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  movies[['budget', 'prod_multiple']] = (movies['budget']


,budget,prod_multiple
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,"$23,000,000",(worldwide box office is 1.0 times production ...
9,NaN,NaN


In [412]:
# Finalizing cleaning budget
get_monetary_clean(movies, ['budget'])

### Cleaning `prod_multiple`

In [413]:
#investigating prod_multiple
movies['prod_multiple'].unique()

array([nan, '(worldwide box office is 1.0 times production budget)',
       '(worldwide box office is 6.0 times production budget)',
       '(worldwide box office is 7.7 times production budget)',
       '(worldwide box office is 0.3 times production budget)',
       '(worldwide box office is 2.6 times production budget)',
       '(worldwide box office is 5.0 times production budget)',
       '(worldwide box office is 1.4 times production budget)',
       '(worldwide box office is 3.6 times production budget)',
       '(worldwide box office is 3.0 times production budget)',
       '(worldwide box office is 3.4 times production budget)',
       '(worldwide box office is 3.1 times production budget)',
       '(worldwide box office is 1.5 times production budget)',
       '(worldwide box office is 0.1 times production budget)',
       '(worldwide box office is 2.0 times production budget)',
       '(worldwide box office is 4.9 times production budget)',
       '(worldwide box office is 0.

In [414]:
#Cleaning up prod multiple. 
movies['prod_multiple'] = (movies['prod_multiple']
                           .str.replace('(worldwide box office is ', '', regex=False)
                           .str.replace(' times production budget)', '', regex=False)
                           .str.replace(',', '', regex=False)
                           .astype('float'))

movies['prod_multiple'].unique()

array([      nan, 1.000e+00, 6.000e+00, 7.700e+00, 3.000e-01, 2.600e+00,
       5.000e+00, 1.400e+00, 3.600e+00, 3.000e+00, 3.400e+00, 3.100e+00,
       1.500e+00, 1.000e-01, 2.000e+00, 4.900e+00, 8.000e-01, 1.800e+00,
       4.300e+00, 1.900e+00, 4.000e-01, 1.100e+00, 9.000e-01, 1.300e+00,
       5.800e+00, 2.000e-01, 2.900e+00, 2.400e+00, 4.600e+00, 6.500e+00,
       5.000e-01, 1.200e+00, 2.700e+00, 1.700e+00, 4.800e+00, 7.000e-01,
       9.600e+00, 6.000e-01, 1.490e+01, 5.500e+00, 1.000e+01, 3.300e+00,
       6.800e+00, 5.100e+00, 3.800e+00, 9.700e+00, 2.030e+01, 3.500e+00,
       3.700e+00, 7.100e+00, 6.300e+00, 9.200e+00, 2.100e+00, 7.300e+00,
       7.200e+00, 1.050e+01, 4.500e+00, 5.200e+00, 2.300e+00, 2.200e+00,
       4.200e+00, 5.600e+00, 2.500e+00, 8.960e+01, 2.800e+00, 8.400e+00,
       1.330e+01, 4.700e+00, 7.900e+00, 4.000e+00, 9.900e+00, 3.900e+00,
       5.400e+00, 1.600e+00, 4.315e+02, 6.900e+00, 1.380e+01, 8.700e+00,
       6.700e+00, 1.230e+01, 8.300e+00, 1.340e+01, 

### Cleaning `dom_pct`

In [415]:
# Investigating dom_pct
movies['dom_pct'].unique()

array(['2.1% (domestic box office/worldwide)', nan,
       '65.3% (domestic box office/worldwide)',
       '34.3% (domestic box office/worldwide)',
       '51.6% (domestic box office/worldwide)',
       '9.6% (domestic box office/worldwide)',
       '49.1% (domestic box office/worldwide)',
       '77.7% (domestic box office/worldwide)',
       '11.4% (domestic box office/worldwide)',
       '35.2% (domestic box office/worldwide)',
       '47.1% (domestic box office/worldwide)',
       '32.8% (domestic box office/worldwide)',
       '41.9% (domestic box office/worldwide)',
       '0.9% (domestic box office/worldwide)',
       '31.7% (domestic box office/worldwide)',
       '47.9% (domestic box office/worldwide)',
       '21.6% (domestic box office/worldwide)',
       '26.8% (domestic box office/worldwide)',
       '50.4% (domestic box office/worldwide)',
       '50.7% (domestic box office/worldwide)',
       '76.2% (domestic box office/worldwide)',
       '26.4% (domestic box office/wor

In [416]:
# Cleaning up dom_pct
movies['dom_pct'] = (movies['dom_pct']
                     .str.replace('% (domestic box office/worldwide)', '', regex=False)
                     .astype('float')/100)

movies['dom_pct'].head(10)

0    0.021
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8    0.653
9      NaN
Name: dom_pct, dtype: float64

### Cleaning up `prod_method`

In [417]:
#Prod_method
movies['prod_method'].unique()

array(['Live Action', 'Digital Animation', 'Hand Animation',
       'Animation/Live Action', nan, 'Stop-Motion Animation',
       'Rotoscoping', 'Multiple Production Methods'], dtype=object)

In [418]:
set_categorical(movies, ['prod_method'])
movies['prod_method'].head(10)

0          Live Action
1    Digital Animation
2          Live Action
3          Live Action
4       Hand Animation
5          Live Action
6          Live Action
7          Live Action
8          Live Action
9          Live Action
Name: prod_method, dtype: category
Categories (7, object): ['Animation/Live Action', 'Digital Animation', 'Hand Animation', 'Live Action', 'Multiple Production Methods', 'Rotoscoping', 'Stop-Motion Animation']

### Cleaning movie comps

In [419]:
# Investigating comps
movies['comps'].unique()

array(['vs. AvatarCreate your own comparison chart…',
       'Create your own comparison chart…',
       'vs. Kimetsu no Yaiba: Mugen Ressha-HenCreate your own comparison chart…',
       ..., 'vs. Baby BoomCreate your own comparison chart…',
       'vs. Mississippi BurningCreate your own comparison chart…',
       'vs. Casse-tête chinoisCreate your own comparison chart…'],
      dtype=object)

In [420]:
# Comps does not give any interesting information
movies.drop(columns = 'comps', inplace=True)

### Cleaning `opening_wknd_bo`

In [421]:
movies['opening_wknd_bo'].unique()

array(['$119,364 (32.8% of total gross)', nan,
       '$3,104,204 (19.9% of total gross)', ...,
       '$4,216,063 (27.3% of total gross)',
       '$11,260,685 (12.3% of total gross)',
       '$19,558,558 (29.3% of total gross)'], dtype=object)

In [422]:
movies[['opening_wknd_bo', 'opening_wknd_bo_pct_tot']] = movies['opening_wknd_bo'].str.split(' ', 1, expand=True)

/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/1389905330.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  movies[['opening_wknd_bo', 'opening_wknd_bo_pct_tot']] = movies['opening_wknd_bo'].str.split(' ', 1, expand=True)


In [423]:
get_monetary_clean(movies, ['opening_wknd_bo'])

### Cleaning `opening_wknd_bo_pct_tot`

In [424]:
movies['opening_wknd_bo_pct_tot'].unique()

array(['(32.8% of total gross)', nan, '(19.9% of total gross)',
       '(8.6% of total gross)', '(30.7% of total gross)',
       '(28.4% of total gross)', '(37.0% of total gross)',
       '(19.6% of total gross)', '(30.3% of total gross)',
       '(15.5% of total gross)', '(36.7% of total gross)',
       '(34.4% of total gross)', '(28.5% of total gross)',
       '(24.7% of total gross)', '(31.9% of total gross)',
       '(24.9% of total gross)', '(20.3% of total gross)',
       '(55.1% of total gross)', '(43.8% of total gross)',
       '(38.2% of total gross)', '(37.7% of total gross)',
       '(40.5% of total gross)', '(27.3% of total gross)',
       '(25.0% of total gross)', '(29.7% of total gross)',
       '(38.4% of total gross)', '(6.8% of total gross)',
       '(30.0% of total gross)', '(24.8% of total gross)',
       '(27.9% of total gross)', '(34.8% of total gross)',
       '(28.2% of total gross)', '(44.0% of total gross)',
       '(30.5% of total gross)', '(37.1% of total gro

In [425]:
movies['opening_wknd_bo_pct_tot'] = (movies['opening_wknd_bo_pct_tot']
                                     .str.replace('% of total gross)', '', regex=False)
                                     .str.replace('(', '', regex=False)
                                     .astype('float')/100)
movies['opening_wknd_bo_pct_tot'].unique()

array([0.328,   nan, 0.199, 0.086, 0.307, 0.284, 0.37 , 0.196, 0.303,
       0.155, 0.367, 0.344, 0.285, 0.247, 0.319, 0.249, 0.203, 0.551,
       0.438, 0.382, 0.377, 0.405, 0.273, 0.25 , 0.297, 0.384, 0.068,
       0.3  , 0.248, 0.279, 0.348, 0.282, 0.44 , 0.305, 0.371, 0.39 ,
       0.349, 0.473, 0.299, 0.19 , 0.469, 0.359, 0.393, 0.265, 0.637,
       0.36 , 0.233, 0.325, 0.363, 0.006, 0.5  , 0.391, 0.336, 0.406,
       0.444, 0.379, 0.429, 0.496, 0.565, 0.317, 0.343, 0.422, 0.52 ,
       0.017, 0.046, 0.537, 0.084, 0.402, 0.413, 0.432, 0.34 , 0.154,
       0.192, 0.357, 0.283, 0.268, 0.313, 0.02 , 0.137, 1.   , 0.274,
       0.32 , 0.266, 0.159, 0.531, 0.221, 0.185, 0.312, 0.161, 0.138,
       0.368, 0.301, 0.069, 0.111, 0.005, 0.204, 0.013, 0.011, 0.172,
       0.003, 0.366, 0.162, 0.056, 0.193, 0.383, 0.261, 0.188, 0.327,
       0.291, 0.482, 0.271, 0.242, 0.17 , 0.264, 0.15 , 0.146, 0.404,
       0.28 , 0.104, 0.22 , 0.041, 0.053, 0.178, 0.201, 0.106, 0.365,
       0.004, 0.002,

### Cleaning `ww_bo`, `ww_bo`, `int_bo`, `dom_bo`, `est_dom_bluray`, `tot_dom_vid_sales`, `dot_dom_dvd`, `adj_dom_bo`

In [426]:
_money_cols = ['ww_bo', 'int_bo', 'dom_bo', 'est_dom_bluray',
               'tot_dom_vid_sales', 'est_dom_dvd', 'adj_dom_bo']

get_monetary_clean(movies, _money_cols)

movies[_money_cols].tail(10)

,ww_bo,int_bo,dom_bo,est_dom_bluray,tot_dom_vid_sales,est_dom_dvd,adj_dom_bo
9518,NaN,NaN,22998000.0,NaN,NaN,NaN,49974327.0
9519,NaN,NaN,6633976.0,NaN,NaN,NaN,14415533.0
9520,NaN,NaN,51636504.0,NaN,NaN,NaN,112470197.0
9521,NaN,NaN,15432314.0,NaN,NaN,NaN,33534195.0
9522,22493727.0,1783276.0,20710451.0,NaN,NaN,NaN,45109257.0
9523,202000000.0,110542312.0,91457688.0,NaN,NaN,NaN,198963691.0
9524,NaN,NaN,13798030.0,NaN,NaN,NaN,30054134.0
9525,NaN,NaN,7331647.0,NaN,NaN,NaN,15931564.0
9526,66786966.0,120904.0,66666062.0,NaN,NaN,NaN,144909420.0
9527,NaN,NaN,29804169.0,NaN,NaN,NaN,64917869.0


### Cleaning `int_releases`

In [427]:
# investigating international releases. The columns seems incomplete
movies['int_releases'][0]

'January 1st, 2021 (Wide) (Australia)\nJanuary 1st, 2021 (Wide) (New Zealand)\nJuly 15th, 2021 (Wide), released as Город тайн (Russia (CIS))\nNovember 11th, 2021 (Wide), released as Chi è senza peccato - The Dry (Italy)\nNovember 19th, 2021 (Wide), released as Años de sequia (Spain)\n... Show all releases\n'

In [428]:
# Dropping it
movies.drop(columns='int_releases', inplace=True)

### Cleaning `n_theaters`

In [429]:
movies['n_theaters'].unique()


array(['186 opening theaters/186 max. theaters, 2.5 weeks average run per theater',
       nan,
       '1,975 opening theaters/2,018 max. theaters, 9.1 weeks average run per theater',
       ...,
       '1,373 opening theaters/1,373 max. theaters, 1.0 weeks average run per theater',
       '1,833 opening theaters/1,934 max. theaters, 9.4 weeks average run per theater',
       '1,901 opening theaters/1,922 max. theaters, 6.4 weeks average run per theater'],
      dtype=object)

In [430]:
n_theaters_pattern = r'\d{1,3}(?:,\d{3})*(?:\.\d+)?'

numbers_list = []

for string in movies['n_theaters']:
    string = str(string)
    numbers = re.findall(n_theaters_pattern, string)
    numbers_list.append(numbers)

movies['n_theaters'] = numbers_list

In [431]:
for index, row in movies.iterrows():
    values = row['n_theaters']
    length = len(values)

    if length >= 1:
        movies.loc[index, 'n_opening_theaters'] = values[0].replace(',', '')
    if length >= 2:
        movies.loc[index, 'n_max_theaters'] = values[1].replace(',', '')
    if length >= 3:
        movies.loc[index, 'weeks_average_run_theater'] = values[2].replace(
            ',', '')

movies[['n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater']] = movies[[
    'n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater']].astype('float')

In [432]:
movies[['n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater']].describe()

,n_opening_theaters,n_max_theaters,weeks_average_run_theater
count,6732.000000,6732.000000,6731.000000
mean,1616.567588,1781.497623,5.206819
std,1295.669278,1215.319260,4.458015
min,1.000000,1.000000,0.000000
25%,119.000000,746.500000,3.000000
50%,1562.500000,1747.000000,4.400000
75%,2709.250000,2778.000000,6.300000
max,4735.000000,4802.000000,101.200000


In [433]:
movies = movies.drop(columns='n_theaters')

### Creating a `movie_id`

In [434]:
movies = movies.sort_values(by='year')
movies = movies.reset_index()
movies['movie_id'] = movies.index+1

In [435]:
movies = movies.drop(columns='index')

### Cleaning `mpaa`

In [436]:
# Investigating `mpaa`
movies['mpaa'].unique()

array(['Not Rated', 'Not Rated Passed', 'G', ...,
       'PG-13 for some strong language and suggestive material.(Rating bulletin 2778 (cert #54118), 4/12/2023)',
       'PG-13 for intense sequences of violence and action, language and some suggestive material.(Rating bulletin 2778 (cert #54373), 4/12/2023)',
       'PG for action/peril and some scary images.(Rating bulletin 2764 (cert #54208), 1/4/2023)'],
      dtype=object)

In [437]:
movies['mpaa'].str[:5].unique()

array(['Not R', 'G', 'PG-13', 'G Rat', 'PG fo', 'R Rat', 'G (19', 'PG ra',
       'R for', 'PG', 'PG Ra', 'G (ra', 'G 197', 'PG 19', 'PG 20',
       'G 196', 'PG (1', 'R', 'GP', 'M/PG', 'GP Ed', 'R Edi', 'G Edi',
       'PG Re', 'NC-17', 'PG (r', 'PG Ed', nan, 'PG (a', 'R (ap', 'PG Fo',
       'PG on', 'PGPG ', 'R App', 'PG (O', 'R For', 'PG (e', 'PGPG-',
       'R (af', 'R (Ed', 'R(Rat', 'R Adu', 'R R f', 'R on ', 'PG so',
       'Open ', 'G pre', 'Open', 'G for', 'G 201', 'PG br', 'G (ed',
       'PG or', 'G(Rat', 'R  Fo', 'R vio', 'R str', 'R or ', 'R Eng',
       'PG vi', 'PG th', 'R lan'], dtype=object)

In [438]:
#mpaa ratings'G', 'PG-13','NC-17', ' PG', 'R', 'Not Rated'. Until 1972 PG was called GP
mpaa_ratings_pattern =  r'\b(G|PG-13|NC-17|PG|R|Not Rated|GP)\b'

movies['mpaa_clean'] = (movies['mpaa']
                        .str.extract(mpaa_ratings_pattern, expand=False))
movies['mpaa_clean'].unique()

array(['Not Rated', 'G', 'PG-13', 'PG', 'R', 'GP', 'NC-17', nan],
      dtype=object)

In [439]:
# Overwriting GP to PG
movies.loc[movies['mpaa_clean'] == 'GP', 'mpaa_clean'] = 'PG'

In [440]:
# Checking failed mpaa rating extraction
movies.loc[(movies['mpaa_clean'].isna()) & ~(movies['mpaa'].isna()), ['movie_title', 'year', 'mpaa', 'mpaa_clean']]

,movie_title,year,mpaa,mpaa_clean
930,E.T. the Extra-Terrestrial,1982,PGPG for language and mild thematic elements. ...,NaN
1055,Stop Making Sense,1984,Not RatedPG for brief suggestive material.(Rat...,NaN
1198,Rocky IV,1985,PGPG-13 for violent sports action and brief st...,NaN
3199,Requiem for a Dream,2000,Open,NaN


In [441]:
# Since it is big, important movies, we impute the values manually
_impute_mpaa_values = (['E.T. the Extra-Terrestrial', 1982, 'PG'],
                  ['Stop Making Sense',1984, 'PG'],
                  ['Rocky IV', 1985, 'PG-13'],
                  ['Requiem for a Dream', 2000, 'Not Rated'])

In [442]:
for rating in _impute_mpaa_values:
    movies.loc[(movies['movie_title'] == rating[0]) & (movies['year'] == rating[1]), 'mpaa_clean'] = rating[2]

movies.loc[(movies['mpaa_clean'].isna()) & ~(movies['mpaa'].isna()), ['movie_title', 'year', 'mpaa', 'mpaa_clean']]

,movie_title,year,mpaa,mpaa_clean


In [443]:
# Everything seems now resolved
movies =( movies
         .drop(columns='mpaa')
         .rename(columns={'mpaa_clean': 'mpaa'}))
set_categorical(movies, ['mpaa'])
movies['mpaa'].head(25)

0     Not Rated
1     Not Rated
2     Not Rated
3     Not Rated
4     Not Rated
5     Not Rated
6     Not Rated
7     Not Rated
8     Not Rated
9     Not Rated
10    Not Rated
11            G
12        PG-13
13    Not Rated
14            G
15            G
16    Not Rated
17    Not Rated
18            G
19    Not Rated
20    Not Rated
21    Not Rated
22            G
23    Not Rated
24            G
Name: mpaa, dtype: category
Categories (6, object): ['G', 'NC-17', 'Not Rated', 'PG', 'PG-13', 'R']

### Cleaning `keywords`

In [444]:
movies['keywords'].head(10)

0    Whitewashed Casting, Ku Klux Klan, Historical ...
1                                                  NaN
2                                                  NaN
3    War Drama, World War I, Anti-war, Romance, Rel...
4                                                  NaN
5                                                  NaN
6                                                  NaN
7       Gothic Horror, Frankenstien's Monster, Monster
8    Nature Horror, Animal Lead, Animals Gone Bad, ...
9                                           Short Film
Name: keywords, dtype: object

In [445]:
#splitting keywords
movies['keywords'] = movies['keywords'].str.split(', ')

In [446]:
movies['keywords'].head(10)

0    [Whitewashed Casting, Ku Klux Klan, Historical...
1                                                  NaN
2                                                  NaN
3    [War Drama, World War I, Anti-war, Romance, Re...
4                                                  NaN
5                                                  NaN
6                                                  NaN
7     [Gothic Horror, Frankenstien's Monster, Monster]
8    [Nature Horror, Animal Lead, Animals Gone Bad,...
9                                         [Short Film]
Name: keywords, dtype: object

### Cleaning `prod_countries`

In [447]:
movies['prod_countries'] = movies['prod_countries'].str.split(', ')
movies['prod_countries'].tail(25)

9501            [United States]
9502            [United States]
9503       [Russian Federation]
9504            [United States]
9505    [Spain, United Kingdom]
9506                    [Japan]
9507                    [Spain]
9508                        NaN
9509        [Republic of Korea]
9510            [United States]
9511                    [Japan]
9512                    [China]
9513                    [India]
9514                    [China]
9515           [United Kingdom]
9516                    [China]
9517            [United States]
9518            [United States]
9519            [United States]
9520            [United States]
9521                    [China]
9522                    [China]
9523            [United States]
9524                    [China]
9525                    [China]
Name: prod_countries, dtype: object

### Cleaning `movie_title`

In [448]:
movies['movie_title'] = movies['movie_title'].astype('string')

In [449]:
movies['movie_title'] = movies['movie_title'] + ' (' + movies['year'].astype('str') + ')'

### Cleaning `crew`

In [450]:
movies['crew'].tail()

9521                 [['Yuanhan Yang (杨沅翰)', 'Director']]
9522                   [['Zhuqing Ji (季竹青)', 'Director']]
9523    [['Rob Marshall', 'Director'], ['David Magee',...
9524                  [['Xiaoshi Liu (刘晓世)', 'Director']]
9525                       [['Yan Han (韩延)', 'Director']]
Name: crew, dtype: object

In [451]:
# Turning into its original list form
movies['crew'] = movies['crew'].apply(eval)
movies['crew'][0]

[['D.W. Griffith', 'Director'],
 ['D.W. Griffith', 'Screenwriter'],
 ['Frank E. Woods', 'Screenwriter']]

In [452]:
# Turning list to dictionary
for index, row in enumerate(movies['crew']):
    crew_dict = {}
    for person in row:
        key, value = person[1], person[0]
        if key in crew_dict:
            crew_dict[key].append(value)
        else:
            crew_dict[key] = [value]
    movies.loc[index, 'crew'] = [copy.deepcopy(crew_dict)]

movies['crew'][0]

[{'Director': ['D.W. Griffith'],
  'Screenwriter': ['D.W. Griffith', 'Frank E. Woods']}]

In [453]:
# Replace and combine all 'based on' into one key
based_on_pattern = r'(?i).*\bbased\b.*'
replacement = 'Based on'

for index, row in movies.iterrows():
    new_dict = {}
    dicts = row['crew']
    for orig_key, orig_value in dicts[0].items():
        key = re.sub(based_on_pattern, replacement, orig_key)
        value = orig_value
        if key in new_dict:
            new_dict[key].append(value)
        else:
            new_dict[key] = value
    movies.loc[index, 'crew'] = [copy.deepcopy(new_dict)]

In [454]:
crew_top_roles(0,50)

564
Director                      8179
Screenwriter                  5785
Producer                      4334
Composer                      3886
Executive Producer            3473
Editor                        2890
Production Designer           2522
Director of Photography       2472
Costume Designer              1524
Story Creator                 1038
Co-Producer                   1014
Casting Director               859
Based on                       763
Music Supervisor               531
Cinematographer                403
Story by                       373
Associate Producer             222
Visual Effects Supervisor      210
Line Producer                  165
Co-Executive Producer          133
Sound Designer                 126
Art Director                   118
First Assistant Director       103
Character Creator               99
Casting                         88
Sound Mixer                     86
Unit Production Manager         77
Make-up and Hair Designer       72
Production Manag

In [455]:
# Create positional columns
_extract_to_columns = ['Director', 'Producer', 'Executive Producer', 'Editor', 'Composer', 'Screenwriter',
                       'Production Designer', 'Costume Designer', 'Story Creator', 'Director of Photography', 'Based on', 'Casting Director']

movies[_extract_to_columns] = ''


In [456]:
# assigning most common ones to respective rows
# Clean up dictionary keys
# Create columns for: 'Director', 'Producer', 'Casting Director', 'Executive Producer', 'Editor', 'Composer', VFX Supervisor

_position_cleanup = {'Co-Producer': 'Producer',
                     'Screen Story by': 'Story Creator',
                     'Screen Story': 'Story Creator',
                     'Story by': 'Story Creator',
                     'Cinematographer': 'Director of Photography',
                     'Associate Producer': 'Producer',
                     'Co-Executive Producer': 'Executive Producer',
                     'Casting': 'Casting Director',
                     'Co-Director': 'Director',
                     'Film Editor': 'Editor',
                     'Casting by': 'Casting Director',
                     'Casting By': 'Casting Director',
                     'Written By': 'Screenwriter',
                     'Written by': 'Screenwriter',
                     'Co-Editor': 'Editor',
                     'Screen story by': 'Story Creator',
                     'Original Score': 'Composer',
                     'Screenplay by': 'Screenwriter',
                     'Screenplay and Screen Story by': 'Screenwriter',
                     'Story from the Comic "Marmaduke" Created by': 'Based on',
                     'Novel': 'Based on',
                     'Director of Photograpy': 'Director of Photography',
                     'Score': 'Composer',
                     'Story By': 'Story Creator',
                     'Director Of Photography': 'Director of Photography',
                     
                     
                     }

for index, row in movies.iterrows():
    dicts = row['crew']
    copy_dict = copy.deepcopy(dicts[0])
    # Create a copy of the dictionary
    for key, value in _position_cleanup.items():
        if key in copy_dict:
            dicts[0][value] = dicts[0].pop(key)

In [457]:
for index, row in movies.iterrows():
    dicts = row['crew']
    copy_dict = copy.deepcopy(dicts[0])
    for key in _extract_to_columns:
        value = copy_dict.get(key)
        if value is not None:
            movies.at[index, key] = value
            dicts[0].pop(key)

In [458]:
movies.tail()

,url,cast,crew,opening_wknd_bo,legs,dom_pct,adj_dom_bo,budget,dom_releases,video_releases,...,Executive Producer,Editor,Composer,Screenwriter,Production Designer,Costume Designer,Story Creator,Director of Photography,Based on,Casting Director
9521,https://www.the-numbers.com/movie/Qing-Bie-Xia...,[],[{}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
9522,https://www.the-numbers.com/movie/Zhe-Me-Duo-N...,[],[{}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,
9523,https://www.the-numbers.com/movie/Little-Merma...,"['Daveed Diggs', 'Awkwafina', 'Jacob Tremblay'...","[{'Song Writer': ['Howard Ashman', 'Lin-Manuel...",95578040.0,3.11,0.530,296908134.0,250000000.0,"May 26th, 2023 (Wide) by Walt DisneyMay 26th, ...","July 25th, 2023 by Walt Disney Home Entertainment",...,[Jeffrey Silver],[Wyatt Smith],[Alan Menken],[David Magee],,,"[Ron Clements, Hans Christian Andersen]",[Dion Beebe],,
9524,https://www.the-numbers.com/movie/Chang-Kong-Z...,[],[{}],132247.0,2.09,0.002,276743.0,NaN,"April 28th, 2023 (Limited) by Well Go USA, rel...",NaN,...,,,,,,,,,,
9525,https://www.the-numbers.com/movie/Wo-Ai-Ni-(20...,[],[{}],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,,,,,,,,,


In [459]:
movies = movies.drop(columns='crew')

### Cleaning `cast`

In [460]:
movies['cast'] = movies['cast'].apply(eval)

### Cleaning `dom_releases`

In [461]:
movies['dom_releases']

0                               February 8th, 1915 (Wide)
1                              December 24th, 1916 (Wide)
2                                  March 6th, 1921 (Wide)
3                    November 19th, 1925 (Limited) by MGM
4       December 30th, 1925 (Wide) by MGM, released as...
                              ...                        
9521                                                  NaN
9522                                                  NaN
9523    May 26th, 2023 (Wide) by Walt DisneyMay 26th, ...
9524    April 28th, 2023 (Limited) by Well Go USA, rel...
9525                                                  NaN
Name: dom_releases, Length: 9526, dtype: object

In [462]:
release_pattern = r'^([A-Za-z ]+ \d{1,2}(?:st|nd|rd|th), \d{4})'

movies['dom_release_date'] = (movies['dom_releases']
                              .str.extract(release_pattern, expand=True))
movies[['dom_releases', 'dom_release_date']]

,dom_releases,dom_release_date
0,"February 8th, 1915 (Wide)","February 8th, 1915"
1,"December 24th, 1916 (Wide)","December 24th, 1916"
2,"March 6th, 1921 (Wide)","March 6th, 1921"
3,"November 19th, 1925 (Limited) by MGM","November 19th, 1925"
4,"December 30th, 1925 (Wide) by MGM, released as...","December 30th, 1925"
...,...,...
9521,NaN,NaN
9522,NaN,NaN
9523,"May 26th, 2023 (Wide) by Walt DisneyMay 26th, ...","May 26th, 2023"
9524,"April 28th, 2023 (Limited) by Well Go USA, rel...","April 28th, 2023"


In [463]:
# Checking for potentially unsuccessful extractions
movies.loc[(movies['dom_release_date'].isna())&~(movies['dom_releases'].isna()), ['dom_release_date', 'dom_releases']]

,dom_release_date,dom_releases
124,NaN,"1955 (Wide), released as Galapagos (1955)"
293,NaN,"1966 (Wide) by Allied Artists, released as A M..."
315,NaN,1967 (Wide)
336,NaN,1968 (Wide)
338,NaN,1968 (Wide) by Sony Pictures
...,...,...
8941,NaN,2020 (Canceled) by Walt Disney
8991,NaN,"2020 (Canceled) by Focus FeaturesJuly 30th, 20..."
9114,NaN,2021 (Canceled) by Sony Pictures
9124,NaN,2021 (Canceled) by Well Go USA


In [464]:
# Release year only is not interesting, already included in years, hence leaving it fully off.
movies = movies.drop(columns='dom_releases')

In [465]:
ordinal_pattern = r'(?<=\d)(?:st|nd|rd|th)'
movies['dom_release_date'] = (movies['dom_release_date']
                              .apply(lambda x: re.sub(ordinal_pattern, '', str(x))))

set_as_date(movies, 'dom_release_date', '%B %d, %Y')
movies['dom_release_date'].tail()

9521          NaT
9522          NaT
9523   2023-05-26
9524   2023-04-28
9525          NaT
Name: dom_release_date, dtype: datetime64[ns]

### Cleaning `video_release_date`

In [466]:
movies['video_release_date'] = (movies['video_releases']
                                .str.extract(release_pattern, expand=True))
movies.loc[movies['video_releases'].isna(), ['video_releases', 'video_release_date']].sum()

video_releases        0
video_release_date    0
dtype: object

In [467]:
movies['video_release_date'] = (movies['video_release_date']
                                .apply(lambda x: re.sub(ordinal_pattern, '', str(x))))
set_as_date(movies, 'video_release_date', '%B %d, %Y')
movies['video_release_date'].tail()

9521          NaT
9522          NaT
9523   2023-07-25
9524          NaT
9525          NaT
Name: video_release_date, dtype: datetime64[ns]

In [468]:
movies.loc[(~movies['video_releases'].isna()) & (movies['video_release_date'].isna()), ['video_releases', 'video_release_date']]

,video_releases,video_release_date
430,1980s by Paramount Home Video,NaT
432,"1974 by ABC, released as The Valachi Papers",NaT
549,1982 by CBS,NaT
692,1983 by ABC,NaT
755,2007 by Warner Home Video,NaT
779,1980 by MGM Video,NaT
931,1982 by Paramount Home Video,NaT
1056,"1995 by Columbia Pictures Home Entertainment, ...",NaT
1074,1985 by MGM Video,NaT
1171,1985 by Sony Pictures Home Entertainment,NaT


In [469]:
# Since it is only very few and only give the full year, we can go ahead and ignore it and drop the column
movies = movies.drop(columns='video_releases')

### Cleaning `url`

In [470]:
movies['url'] = movies['url'].astype('string')

### Cleaning `languages`

In [471]:
movies['languages'].tail(25)

9501                                     English
9502                                     English
9503                                     Russian
9504                                     English
9505                                     English
9506                                    Japanese
9507                                     Spanish
9508                                     English
9509                                    Mandarin
9510                                     English
9511                                    Japanese
9512                                    Mandarin
9513    Hindi, Kannada, Malayalam, Tamil, Telugu
9514                                    Mandarin
9515                                     English
9516                                    Mandarin
9517                                     English
9518                                     English
9519                                     English
9520                                     English
9521                

In [472]:
movies['languages'] = movies['languages'].str.split(',')
movies['languages'].tail(25)

9501                                         [English]
9502                                         [English]
9503                                         [Russian]
9504                                         [English]
9505                                         [English]
9506                                        [Japanese]
9507                                         [Spanish]
9508                                         [English]
9509                                        [Mandarin]
9510                                         [English]
9511                                        [Japanese]
9512                                        [Mandarin]
9513    [Hindi,  Kannada,  Malayalam,  Tamil,  Telugu]
9514                                        [Mandarin]
9515                                         [English]
9516                                        [Mandarin]
9517                                         [English]
9518                                         [English]
9519      

### Cleaning producers

In [473]:
movies['producers'].value_counts()[:50]

Warner Bros.                                           50
Universal Pictures                                     29
Paramount Pictures                                     29
Columbia Pictures                                      28
20th Century Fox                                       23
Castle Rock Entertainment                              23
Working Title Films                                    22
Regency Enterprises                                    22
Twentieth Century Fox                                  21
Marvel Studios                                         21
Walt Disney Pictures                                   20
Tri-Star Pictures                                      18
DreamWorks Animation                                   18
Touchstone Pictures                                    17
Metro-Goldwyn-Mayer Pictures                           15
Walt Disney Animation Studios                          14
Davis Entertainment                                    13
Lucasfilm     

In [474]:
movies['producers'] = movies['producers'].str.split(',')

In [475]:
movies['franchise'].unique()

array([nan, 'King Kong', 'Wizard of Oz', 'Fantasia', 'Bambi',
       "Father Chuck O'Malley", 'Cinderella', 'Peter Pan',
       'Lady and the Tramp', 'Around the World in 80 Days',
       'Peyton Place', 'Psycho', '101 Dalmatians', 'Guns of Navarone',
       'Eddie Felson', 'James Bond', 'Mary Poppins', 'Pink Panther',
       'Flint', "Donald Hamilton's Matt Helm", 'Un homme et une femme',
       'Dollar Trilogy', 'Jungle Book', 'Billy Jack',
       'Planet of the Apes', 'Odd Couple',
       "George A. Romero's Dead Series", '2001', 'Thomas Crown Affair',
       'Easy Rider', 'Butch Cassidy and the Sundance Kid', 'Herbie',
       'Airport', 'Dirty Harry', 'French Connection', 'Last Picture Show',
       'Shaft', 'Willard', 'Godfather', 'Exorcist', 'The Sting',
       'American Graffiti', 'Texas Chainsaw Massacre', 'Benji',
       'Death Wish', 'Chinatown', 'Monty Python', 'Jaws', 'Carrie',
       'Rocky', 'Bad News Bears', 'Omen', 'Winnie the Pooh', 'Star Wars',
       'Smokey and the 

In [476]:
# Investigating shows that multiple keywords seem to be together in camelCase
# Example: 'Man of SteelSupermanBatmanDC Extended Universe'

franchise_pattern = r'(?<=[a-z])(?=[A-Z])'

movies['franchise_clean'] = (movies['franchise']
                             .apply(lambda x: re.split(franchise_pattern, str(x)) if str(x) != 'nan' else None))
movies.loc[~movies['franchise'].isna(), ['franchise', 'franchise_clean']][:50]

,franchise,franchise_clean
8,King Kong,[King Kong]
12,Wizard of Oz,[Wizard of Oz]
15,Fantasia,[Fantasia]
22,Bambi,[Bambi]
33,Father Chuck O'Malley,[Father Chuck O'Malley]
42,Father Chuck O'Malley,[Father Chuck O'Malley]
77,Cinderella,[Cinderella]
97,Peter Pan,[Peter Pan]
127,Lady and the Tramp,[Lady and the Tramp]
135,Around the World in 80 Days,[Around the World in 80 Days]


In [477]:
#Looks fine
movies['franchise'] = movies['franchise_clean']
movies = movies.drop(columns='franchise_clean')

## Cleaning `df_bo`

In [478]:
df_bo.head()

,Unnamed: 0,id,date,rank,gross,pct_change,theaters,per_theater,total_gross,kind,kind_num,pct_lw
0,0,https://www.the-numbers.com/movie/Cinderella-(...,"Dec 18, 1981",6,"$1,300,000",,560,"$2,321","$1,300,000",weekend,"1,662",NaN
1,1,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 1, 1982",3,"$4,691,990",,"1,073","$4,373","$17,223,729",weekend,"1,664",NaN
2,2,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 8, 1982",8,"$1,774,588",-62%,"1,050","$1,690","$19,966,217",weekend,"1,665",NaN
3,3,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 15, 1982",9,"$1,235,249",-30%,940,"$1,314","$4,846,893",weekend,"1,666",NaN
4,4,https://www.the-numbers.com/movie/Cinderella-(...,"Nov 20, 1987",2,"$5,165,881",,"1,646","$3,138","$5,165,881",weekend,"1,971",NaN


In [479]:
df_bo['market'] = 'Domestic'

In [480]:
df_bo.shape

(431954, 13)

In [481]:
df_country['market'] = df_country['id'].str.split('/').str.get(-1).str.split('#').str.get(0).str.replace('-', ' ')

df_country['id'] = df_country['id'].str.replace(str('/' + df_country['id'].str.split('/').str.get(-1)), '', regex=False)

In [482]:
df_country['id'] = df_country['id'].str.extract(r"(https://www\.the-numbers\.com/movie/[^\s/]+)")

In [483]:
df_country.shape

(569777, 13)

In [484]:
df_country.duplicated().sum()

284613

In [485]:
df_country = df_country.drop_duplicates()

In [486]:
df_bo = pd.concat([df_bo, df_country])

In [487]:
df_bo.shape

(717118, 13)

In [488]:
df_bo = df_bo.drop(columns=['Unnamed: 0'])
df_bo = df_bo.rename(columns={'id': 'url'})

In [489]:
df_bo.shape

(717118, 12)

In [490]:
# Adding additional movie info
_df = movies[['movie_id', 'url', 'year', 'movie_title']]
bo = pd.merge(df_bo, _df, on='url', how='left')

In [491]:
df_bo.shape

(717118, 12)

In [492]:
bo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717118 entries, 0 to 717117
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   url          717118 non-null  object
 1   date         717118 non-null  object
 2   rank         717118 non-null  object
 3   gross        717118 non-null  object
 4   pct_change   717118 non-null  object
 5   theaters     717118 non-null  object
 6   per_theater  717118 non-null  object
 7   total_gross  717118 non-null  object
 8   kind         717118 non-null  object
 9   kind_num     717118 non-null  object
 10  pct_lw       236377 non-null  object
 11  market       717118 non-null  object
 12  movie_id     717118 non-null  int64 
 13  year         717118 non-null  int64 
 14  movie_title  717118 non-null  string
dtypes: int64(2), object(12), string(1)
memory usage: 87.5+ MB


### Cleaning `gross`, `per_theater`, `total_gross`

In [493]:
# Cleaning monetary values
get_monetary_clean(bo, ['gross', 'per_theater', 'total_gross'])

### Cleaning `bo_url`

In [494]:
bo['url'] = bo['url'].astype('string')

### Cleaning `date`

In [495]:
bo.head()

,url,date,rank,gross,pct_change,theaters,per_theater,total_gross,kind,kind_num,pct_lw,market,movie_id,year,movie_title
0,https://www.the-numbers.com/movie/Cinderella-(...,"Dec 18, 1981",6,1300000.0,,560,2321.0,1300000.0,weekend,"1,662",NaN,Domestic,78,1950,Cinderella (1950)
1,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 1, 1982",3,4691990.0,,"1,073",4373.0,17223729.0,weekend,"1,664",NaN,Domestic,78,1950,Cinderella (1950)
2,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 8, 1982",8,1774588.0,-62%,"1,050",1690.0,19966217.0,weekend,"1,665",NaN,Domestic,78,1950,Cinderella (1950)
3,https://www.the-numbers.com/movie/Cinderella-(...,"Jan 15, 1982",9,1235249.0,-30%,940,1314.0,4846893.0,weekend,"1,666",NaN,Domestic,78,1950,Cinderella (1950)
4,https://www.the-numbers.com/movie/Cinderella-(...,"Nov 20, 1987",2,5165881.0,,"1,646",3138.0,5165881.0,weekend,"1,971",NaN,Domestic,78,1950,Cinderella (1950)


In [496]:
bo['date'] = pd.to_datetime(bo['date'])

### Creating day_kind

In [497]:
bo.columns

Index(['url', 'date', 'rank', 'gross', 'pct_change', 'theaters', 'per_theater',
       'total_gross', 'kind', 'kind_num', 'pct_lw', 'market', 'movie_id',
       'year', 'movie_title'],
      dtype='object')

In [498]:
bo.shape

(717118, 15)

In [499]:
def classify_date(date):
    # Check if the date is a business day
    if date.weekday() < 4:
        return "business"
    
    # Check if the date is a weekend day
    elif date.weekday() >= 4:
        return "weekend"
    
    # Otherwise, classify it as a holiday
    return "holiday"

# Precompute the set of holidays in the United States
us_holidays = holidays.UnitedStates()

# Apply the classification function to the date column
bo['day_kind'] = bo.loc[(bo['market'] == 'Domestic') & (bo['kind'] == 'daily'), 'date'].apply(
    lambda date: classify_date(date) if date not in us_holidays else "Holiday"
)

### Cleaning `rank`, `kind_num`

In [500]:
bo['rank'].unique()

array([6, 3, 8, 9, 2, 4, 7, 11, 12, 20, 5, 85, '19', '27', '55', '20',
       '26', '60', '13', '21', '35', '56', '59', '66', '80', '22', '77',
       '14', '36', '62', '82', '1', '5', '9', '17', '25', '42', '3', '8',
       '4', '7', '28', '43', '11', '18', '32', '45', '63', '71', '86',
       '51', '72', '2', '6', '10', '34', '65', '73', '74', '16', '37',
       '47', '53', '46', '12', '15', '38', '57', '54', '50', '41', '40',
       '44', '33', '61', '68', '69', '83', '90', '104', '67', '70', '81',
       '89', '101', '30', '24', '31', '49', '39', '29', '75', '23', '78',
       '58', '64', '52', '76', '98', '79', '-', '110', '112', '48', '94',
       '95', '88', '92', '93', '91', '85', '87', '84', '97', '107', '117',
       '102', '114', '96', '103', '105', '111', '118', '113', '125',
       '119', '108', '106', '109', '100', '99', '120', '115', '116',
       '133', '122', '121', '134', '130', '139', '136', 'P', '123', '127',
       18, 23, 29, 37, 38, 52, 39, 13, 21, 33, 41, 50, 48

In [501]:
bo['rank'] = pd.to_numeric(bo['rank'], errors='coerce', downcast='integer')
bo['kind_num'] = pd.to_numeric(bo['kind_num'], errors='coerce', downcast='integer')
bo[['kind_num', 'rank']]

,kind_num,rank
0,NaN,6.0
1,NaN,3.0
2,NaN,8.0
3,NaN,9.0
4,NaN,2.0
...,...,...
717113,1.0,24.0
717114,NaN,NaN
717115,1.0,NaN
717116,109.0,NaN


In [502]:
bo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717118 entries, 0 to 717117
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   url          717118 non-null  string        
 1   date         717118 non-null  datetime64[ns]
 2   rank         479279 non-null  float64       
 3   gross        717118 non-null  float64       
 4   pct_change   717118 non-null  object        
 5   theaters     717118 non-null  object        
 6   per_theater  691982 non-null  float64       
 7   total_gross  717118 non-null  float64       
 8   kind         717118 non-null  object        
 9   kind_num     711820 non-null  float64       
 10  pct_lw       236377 non-null  object        
 11  market       717118 non-null  object        
 12  movie_id     717118 non-null  int64         
 13  year         717118 non-null  int64         
 14  movie_title  717118 non-null  string        
 15  day_kind     258147 non-null  obje

### Cleaning `kind`

In [503]:
set_categorical(bo, ['kind'])

### Cleaning `pct_change`, `pct_lw`

In [504]:
bo ['pct_change'] = pd.to_numeric(bo['pct_change'].str.replace('%', '', regex=False).str.replace('\xa0', '', regex=False), errors='coerce', downcast='float')/100
bo ['pct_lw'] = pd.to_numeric(bo['pct_lw'].str.replace('%', '', regex=False).str.replace('\xa0', '', regex=False), errors='coerce', downcast='float')/100

### Cleanin `theaters`

In [505]:
bo['theaters'] = pd.to_numeric(bo['theaters'], errors='coerce', downcast='integer')

## Data Exploration

In [506]:
bo.head(10)

,url,date,rank,gross,pct_change,theaters,per_theater,total_gross,kind,kind_num,pct_lw,market,movie_id,year,movie_title,day_kind
0,https://www.the-numbers.com/movie/Cinderella-(...,1981-12-18,6.0,1300000.0,NaN,560.0,2321.0,1300000.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
1,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-01,3.0,4691990.0,NaN,NaN,4373.0,17223729.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
2,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-08,8.0,1774588.0,-0.62,NaN,1690.0,19966217.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
3,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-15,9.0,1235249.0,-0.30,940.0,1314.0,4846893.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
4,https://www.the-numbers.com/movie/Cinderella-(...,1987-11-20,2.0,5165881.0,NaN,NaN,3138.0,5165881.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
5,https://www.the-numbers.com/movie/Cinderella-(...,1987-11-27,2.0,7243653.0,0.40,NaN,4372.0,14652555.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
6,https://www.the-numbers.com/movie/Cinderella-(...,1987-12-04,4.0,3024737.0,-0.58,NaN,1825.0,18790781.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
7,https://www.the-numbers.com/movie/Cinderella-(...,1987-12-11,7.0,1690152.0,-0.44,NaN,1103.0,21186597.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
8,https://www.the-numbers.com/movie/Cinderella-(...,1987-12-18,11.0,1093603.0,-0.35,NaN,839.0,22716888.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN
9,https://www.the-numbers.com/movie/Cinderella-(...,1987-12-25,12.0,1623023.0,0.48,NaN,1051.0,25559696.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN


### Generating `weekday`

In [507]:
bo['weekday'] = bo['date'].dt.day_name()

In [508]:
movies['dom_release_date_weekday'] = movies['dom_release_date'].dt.day_name()

### Generating `week_num`

In [509]:
bo['week_num'] = bo['date'].dt.isocalendar().week

In [510]:
movies['dom_release_date_week_um'] = movies['dom_release_date'].dt.isocalendar().week

In [511]:
movies.columns

Index(['url', 'cast', 'opening_wknd_bo', 'legs', 'dom_pct', 'adj_dom_bo',
       'budget', 'runtime', 'keywords', 'source', 'prod_method',
       'creative_type', 'producers', 'prod_countries', 'languages',
       'franchise', 'dom_bo', 'int_bo', 'ww_bo', 'est_dom_dvd',
       'est_dom_bluray', 'tot_dom_vid_sales', 'year', 'movie_title', 'genre',
       'revenue_to_date', 'prod_multiple', 'opening_wknd_bo_pct_tot',
       'n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater',
       'movie_id', 'mpaa', 'Director', 'Producer', 'Executive Producer',
       'Editor', 'Composer', 'Screenwriter', 'Production Designer',
       'Costume Designer', 'Story Creator', 'Director of Photography',
       'Based on', 'Casting Director', 'dom_release_date',
       'video_release_date', 'dom_release_date_weekday',
       'dom_release_date_week_um'],
      dtype='object')

In [512]:
movies['budget'].describe()

count    4.673000e+03
mean     4.146560e+07
std      4.710891e+07
min      2.500000e+04
25%      1.100000e+07
50%      2.500000e+07
75%      5.300000e+07
max      4.600000e+08
Name: budget, dtype: float64

## Estimating Profits

In [513]:
bo.duplicated().sum()

376

In [514]:
#movies['est_profit'] = 

In [515]:
movies.columns

Index(['url', 'cast', 'opening_wknd_bo', 'legs', 'dom_pct', 'adj_dom_bo',
       'budget', 'runtime', 'keywords', 'source', 'prod_method',
       'creative_type', 'producers', 'prod_countries', 'languages',
       'franchise', 'dom_bo', 'int_bo', 'ww_bo', 'est_dom_dvd',
       'est_dom_bluray', 'tot_dom_vid_sales', 'year', 'movie_title', 'genre',
       'revenue_to_date', 'prod_multiple', 'opening_wknd_bo_pct_tot',
       'n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater',
       'movie_id', 'mpaa', 'Director', 'Producer', 'Executive Producer',
       'Editor', 'Composer', 'Screenwriter', 'Production Designer',
       'Costume Designer', 'Story Creator', 'Director of Photography',
       'Based on', 'Casting Director', 'dom_release_date',
       'video_release_date', 'dom_release_date_weekday',
       'dom_release_date_week_um'],
      dtype='object')

### Calculating time difference between `dom_release_date` and `video_release_date`

In [516]:
movies['diff_dom_video_release'] = movies['video_release_date'] - movies['dom_release_date']

In [517]:
bo.head()

,url,date,rank,gross,pct_change,theaters,per_theater,total_gross,kind,kind_num,pct_lw,market,movie_id,year,movie_title,day_kind,weekday,week_num
0,https://www.the-numbers.com/movie/Cinderella-(...,1981-12-18,6.0,1300000.0,NaN,560.0,2321.0,1300000.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN,Friday,51
1,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-01,3.0,4691990.0,NaN,NaN,4373.0,17223729.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN,Friday,53
2,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-08,8.0,1774588.0,-0.62,NaN,1690.0,19966217.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN,Friday,1
3,https://www.the-numbers.com/movie/Cinderella-(...,1982-01-15,9.0,1235249.0,-0.30,940.0,1314.0,4846893.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN,Friday,2
4,https://www.the-numbers.com/movie/Cinderella-(...,1987-11-20,2.0,5165881.0,NaN,NaN,3138.0,5165881.0,weekend,NaN,NaN,Domestic,78,1950,Cinderella (1950),NaN,Friday,47


### Checking if scraped relative values are correct

In [518]:
movies['opening_wknd_bo']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
9521           NaN
9522           NaN
9523    95578040.0
9524      132247.0
9525           NaN
Name: opening_wknd_bo, Length: 9526, dtype: float64

In [519]:
movies['opening_wknd_bo_pct_tot']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
9521      NaN
9522      NaN
9523    0.322
9524    0.478
9525      NaN
Name: opening_wknd_bo_pct_tot, Length: 9526, dtype: float64

## Cleaning `df_int`

In [520]:
df_int.head()

,Unnamed: 0,territory,release_date,opening_wknd,opening_wknd_screens,max_screens,theatrical_engagements,total_bo,report_date,url,country_url,int_total,total_report_date
0,0,Australia,12/11/1943,$0,0.0,0,0,$0,NaN,https://www.the-numbers.com/movie/Bambi-(1942),https://www.the-numbers.com/movie/Bambi-(1942)...,"$165,203,000",NaN
1,0,Australia,12/11/1943,$0,0.0,0,0,$0,NaN,https://www.the-numbers.com/movie/Bambi-(1942),https://www.the-numbers.com/movie/Bambi-(1942)...,"$165,203,000",NaN
0,0,Australia,6/29/2018,$0,0.0,59,69,"$23,510",11/20/2020,https://www.the-numbers.com/movie/Bill-and-Ted...,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",11/20/2020
1,1,New Zealand,9/3/2020,$54,2.0,3,7,"$1,562",9/21/2020,https://www.the-numbers.com/movie/Bill-and-Ted...,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",11/20/2020
2,0,Australia,6/29/2018,$0,0.0,59,69,"$23,510",11/20/2020,https://www.the-numbers.com/movie/Bill-and-Ted...,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",11/20/2020


In [521]:
df_int = df_int.drop(columns=['Unnamed: 0', 'country_url'])

### Cleaning up `release_date`, `report_date`, `total_report_date`

In [522]:
for col in ['release_date', 'report_date', 'total_report_date']:
    df_int[col] = pd.to_datetime(df_int[col], errors='coerce')

### Cleaning up duplicates

In [523]:
df_int.duplicated().sum()

57240

### Cleaning up Index

In [524]:
df_int = df_int.reset_index().drop(columns=['index'])

In [525]:
df_int

,territory,release_date,opening_wknd,opening_wknd_screens,max_screens,theatrical_engagements,total_bo,report_date,url,int_total,total_report_date
0,Australia,1943-12-11,$0,0.0,0,0,$0,NaT,https://www.the-numbers.com/movie/Bambi-(1942),"$165,203,000",NaT
1,Australia,1943-12-11,$0,0.0,0,0,$0,NaT,https://www.the-numbers.com/movie/Bambi-(1942),"$165,203,000",NaT
2,Australia,2018-06-29,$0,0.0,59,69,"$23,510",2020-11-20,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",2020-11-20
3,New Zealand,2020-09-03,$54,2.0,3,7,"$1,562",2020-09-21,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",2020-11-20
4,Australia,2018-06-29,$0,0.0,59,69,"$23,510",2020-11-20,https://www.the-numbers.com/movie/Bill-and-Ted...,"$25,072",2020-11-20
...,...,...,...,...,...,...,...,...,...,...,...
114475,Spain,2013-02-22,"$149,886",55,55,262,"$531,661",2015-12-14,https://www.the-numbers.com/movie/Blue-Valentine,"$6,828,348",2020-06-17
114476,Rest of World,NaT,NaN,NaN,NaN,NaN,"$6,193,817",NaT,https://www.the-numbers.com/movie/Blue-Valentine,"$6,828,348",2020-06-17
114477,South Korea,2012-05-31,$0,0,1,1,"$102,870",2020-06-17,https://www.the-numbers.com/movie/Blue-Valentine,"$6,828,348",2020-06-17
114478,Spain,2013-02-22,"$149,886",55,55,262,"$531,661",2015-12-14,https://www.the-numbers.com/movie/Blue-Valentine,"$6,828,348",2020-06-17


### Cleaning up `opening_wknd`, `total_bo`, `int_total`

In [526]:
get_monetary_clean(df_int, ['opening_wknd', 'total_bo', 'int_total'])

In [527]:
df_int.tail(20)

,territory,release_date,opening_wknd,opening_wknd_screens,max_screens,theatrical_engagements,total_bo,report_date,url,int_total,total_report_date
114460,Brazil,2011-01-21,0.0,0,5,8,6458734.0,2015-12-12,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114461,Bulgaria,2011-02-11,0.0,0,9,9,216191.0,2018-12-29,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114462,Rest of World,NaT,NaN,NaN,NaN,NaN,97853754.0,NaT,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114463,Brazil,2011-01-21,0.0,0,5,8,6458734.0,2015-12-12,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114464,Bulgaria,2011-02-11,0.0,0,9,9,216191.0,2018-12-29,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114465,Rest of World,NaT,NaN,NaN,NaN,NaN,97853754.0,NaT,https://www.the-numbers.com/movie/Yogi-Bear,104528679.0,2018-12-29
114466,Australia,2011-01-26,0.0,0,3,3,0.0,NaT,https://www.the-numbers.com/movie/True-Grit-(2...,81033923.0,NaT
114467,Australia,2011-01-26,0.0,0,3,3,0.0,NaT,https://www.the-numbers.com/movie/True-Grit-(2...,81033923.0,NaT
114468,Nigeria,2010-12-24,0.0,0,"17,386","17,386",87077.0,2018-12-29,https://www.the-numbers.com/movie/Gullivers-Tr...,189238587.0,2018-12-29
114469,Rest of World,NaT,NaN,NaN,NaN,NaN,189151510.0,NaT,https://www.the-numbers.com/movie/Gullivers-Tr...,189238587.0,2018-12-29


### Cleaning `opening_wknd_screens`

In [528]:
df_int['opening_wknd_screens'].unique()

array([0.0, 2.0, 9.0, ..., '21,691', '30,479', '29,996'], dtype=object)

In [529]:
for col in ['opening_wknd_screens', 'max_screens', 'theatrical_engagements']:
    df_int[col] = df_int[col].str.replace(',', '', regex=True)

In [530]:
df_int['opening_wknd_screens'].unique()

array([nan, '14', '0', ..., '21691', '30479', '29996'], dtype=object)

In [531]:
df_int['opening_wknd_screens'] = df_int['opening_wknd_screens'].astype('float')

In [532]:
df_int

,territory,release_date,opening_wknd,opening_wknd_screens,max_screens,theatrical_engagements,total_bo,report_date,url,int_total,total_report_date
0,Australia,1943-12-11,0.0,NaN,NaN,NaN,0.0,NaT,https://www.the-numbers.com/movie/Bambi-(1942),165203000.0,NaT
1,Australia,1943-12-11,0.0,NaN,NaN,NaN,0.0,NaT,https://www.the-numbers.com/movie/Bambi-(1942),165203000.0,NaT
2,Australia,2018-06-29,0.0,NaN,59,69,23510.0,2020-11-20,https://www.the-numbers.com/movie/Bill-and-Ted...,25072.0,2020-11-20
3,New Zealand,2020-09-03,54.0,NaN,3,7,1562.0,2020-09-21,https://www.the-numbers.com/movie/Bill-and-Ted...,25072.0,2020-11-20
4,Australia,2018-06-29,0.0,NaN,59,69,23510.0,2020-11-20,https://www.the-numbers.com/movie/Bill-and-Ted...,25072.0,2020-11-20
...,...,...,...,...,...,...,...,...,...,...,...
114475,Spain,2013-02-22,149886.0,55.0,55,262,531661.0,2015-12-14,https://www.the-numbers.com/movie/Blue-Valentine,6828348.0,2020-06-17
114476,Rest of World,NaT,NaN,NaN,NaN,NaN,6193817.0,NaT,https://www.the-numbers.com/movie/Blue-Valentine,6828348.0,2020-06-17
114477,South Korea,2012-05-31,0.0,0.0,1,1,102870.0,2020-06-17,https://www.the-numbers.com/movie/Blue-Valentine,6828348.0,2020-06-17
114478,Spain,2013-02-22,149886.0,55.0,55,262,531661.0,2015-12-14,https://www.the-numbers.com/movie/Blue-Valentine,6828348.0,2020-06-17


# Charts & Elements for Dashboard

In [533]:
bo.to_pickle('processed_data/bo.pkl.gz')
movies.to_pickle('processed_data/movies.pkl.gz')

In [534]:
bo.shape

(717118, 18)

In [535]:
df_int.duplicated()

0         False
1          True
2         False
3         False
4          True
          ...  
114475    False
114476    False
114477     True
114478     True
114479     True
Length: 114480, dtype: bool

In [536]:
above_10 = bo.loc[(bo['day_kind'] == 'business') & (bo['rank'] == 2) &(bo['kind_num'] > 3) & (bo['weekday'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday'])) & (bo['gross'] > 10_000_000), ['movie_title', 'gross', 'date', 'weekday', 'year', 'day_kind']].sort_values('gross', ascending=False)
above_10

,movie_title,gross,date,weekday,year,day_kind
39572,Sing (2016),17515525.0,2016-12-27,Tuesday,2016,business
408433,Jumanji: Welcome to the Jungle (2017),17073421.0,2017-12-26,Tuesday,2017,business
320741,Avatar (2009),16445291.0,2009-12-23,Wednesday,2009,business
39573,Sing (2016),15501840.0,2016-12-28,Wednesday,2016,business
147762,Frozen (2013),15161249.0,2013-11-27,Wednesday,2013,business
408434,Jumanji: Welcome to the Jungle (2017),15122619.0,2017-12-27,Wednesday,2017,business
39574,Sing (2016),15039855.0,2016-12-29,Thursday,2016,business
408435,Jumanji: Welcome to the Jungle (2017),14841305.0,2017-12-28,Thursday,2017,business
72608,Incredibles 2 (2018),13719714.0,2018-06-26,Tuesday,2018,business
51102,Inside Out (2015),13038808.0,2015-06-23,Tuesday,2015,business


In [537]:
above_10_count = above_10.groupby('movie_title').count()
above_10_count.loc[above_10_count['gross'] > 2,:]

,gross,date,weekday,year,day_kind
movie_title,,,,,
Alvin and the Chipmunks: The Squeakquel (2009),3,3,3,3,3
Incredibles 2 (2018),5,5,5,5,5
Jumanji: Welcome to the Jungle (2017),3,3,3,3,3
Oppenheimer (2023),4,4,4,4,4
Sing (2016),3,3,3,3,3


In [538]:
above_10_no_christmas = bo.loc[(bo['day_kind'] == 'business') & (bo['rank'] == 2) &(bo['kind_num'] > 3) & (bo['weekday'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday'])) & (bo['gross'] > 10_000_000) & (bo['date'].dt.month != 12), ['movie_title', 'gross', 'date', 'weekday', 'year', 'day_kind']].sort_values('gross', ascending=False)
above_10_no_christmas

,movie_title,gross,date,weekday,year,day_kind
147762,Frozen (2013),15161249.0,2013-11-27,Wednesday,2013,business
72608,Incredibles 2 (2018),13719714.0,2018-06-26,Tuesday,2018,business
51102,Inside Out (2015),13038808.0,2015-06-23,Tuesday,2015,business
169797,Oppenheimer (2023),12671950.0,2023-07-24,Monday,2023,business
169798,Oppenheimer (2023),12073530.0,2023-07-25,Tuesday,2023,business
311503,Transformers: Revenge of the Fallen (2009),10939131.0,2009-07-01,Wednesday,2009,business
38940,Fantastic Beasts and Where to Find Them (2016),10931033.0,2016-11-23,Wednesday,2016,business
169799,Oppenheimer (2023),10665845.0,2023-07-26,Wednesday,2023,business
51101,Inside Out (2015),10485640.0,2015-06-22,Monday,2015,business
406608,Justice League (2017),10448116.0,2017-11-22,Wednesday,2017,business


In [539]:
above_10_no_christmas_count = above_10_no_christmas.groupby('movie_title').count()
above_10_no_christmas_count.loc[above_10_no_christmas_count['gross'] > 2,:]

,gross,date,weekday,year,day_kind
movie_title,,,,,
Incredibles 2 (2018),5,5,5,5,5
Oppenheimer (2023),4,4,4,4,4


In [540]:
bo = bo.loc[bo['date'].dt.year <= bo['year'] + 2, :]

In [541]:
filtered_movies = bo.loc[bo['market'] == 'Domestic'].groupby('movie_title').agg(min_rank=('rank', 'min'), max_total_gross=('total_gross', 'max'))

In [542]:
filtered_movies = filtered_movies[filtered_movies['min_rank'] >= 2]

In [543]:
filtered_movies.sort_values('max_total_gross', ascending=False).head(10)

,min_rank,max_total_gross
movie_title,,
Sing (2016),2.0,270329045.0
Alvin and the Chipmunks (2007),2.0,217326974.0
World War Z (2013),2.0,202359711.0
Puss in Boots: The Last Wish (2022),2.0,185535345.0
Lincoln (2012),2.0,182207973.0
The Polar Express (2004),2.0,176530794.0
Dances with Wolves (1990),2.0,175293948.0
Oppenheimer (2023),2.0,174566060.0
The Greatest Showman (2017),2.0,174340174.0


In [544]:
 bo.loc[(bo['kind'] == 'daily') & (bo['movie_title'] == 'Oppenheimer (2023)')].groupby('movie_title').min()

/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/3039827796.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  bo.loc[(bo['kind'] == 'daily') & (bo['movie_title'] == 'Oppenheimer (2023)')].groupby('movie_title').min()


,url,date,rank,gross,pct_change,theaters,per_theater,total_gross,kind_num,pct_lw,market,movie_id,year,day_kind,weekday,week_num
movie_title,,,,,,,,,,,,,,,,
Oppenheimer (2023),https://www.the-numbers.com/movie/Oppenheimer-...,2023-07-20,2.0,10003685.0,-0.45,NaN,2771.0,10500000.0,1.0,-0.59,Domestic,9489,2023,business,Friday,29


In [545]:
movies['opening_v_budget'] = movies['opening_wknd_bo'] / movies['budget']

In [546]:
movies['opening_v_budget']

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
9521         NaN
9522         NaN
9523    0.382312
9524         NaN
9525         NaN
Name: opening_v_budget, Length: 9526, dtype: float64

In [547]:
movies.sort_values('opening_v_budget', ascending=False)[['movie_title', 'budget', 'opening_wknd_bo', 'opening_v_budget']].head(10)

,movie_title,budget,opening_wknd_bo,opening_v_budget
6701,The Gallows (2015),100000.0,9808463.0,98.084630
5663,The Devil Inside (2012),1000000.0,33732515.0,33.732515
6904,Unfriended (2015),1000000.0,15845115.0,15.845115
3700,Tupac Resurrection (2003),300000.0,4632847.0,15.442823
3829,Saw (2004),1200000.0,18276468.0,15.230390
4910,Fireproof (2008),500000.0,6804764.0,13.609528
5221,Paranormal Activity 2 (2010),3000000.0,40678424.0,13.559475
4262,Facing the Giants (2006),100000.0,1343537.0,13.435370
3580,Jackass: The Movie (2002),2000000.0,22763437.0,11.381718
6035,The Purge (2013),3000000.0,34058360.0,11.352787


In [548]:
movies.loc[movies['opening_wknd_bo'] >= 50_000_000, ['movie_title', 'budget', 'opening_wknd_bo', 'opening_v_budget']].sort_values('opening_v_budget', ascending=False).head(10)

,movie_title,budget,opening_wknd_bo,opening_v_budget
5623,Paranormal Activity 3 (2011),5000000.0,52568183.0,10.513637
8280,Halloween (2018),10000000.0,76221545.0,7.622154
8536,Us (2019),20000000.0,71117625.0,3.555881
7506,It (2017),35000000.0,123403419.0,3.525812
3851,The Passion of the Christ (2004),25000000.0,83848082.0,3.353923
8129,A Quiet Place (2018),17000000.0,50203562.0,2.953151
4948,The Twilight Saga: New Moon (2009),50000000.0,142839137.0,2.856783
5219,Jackass 3D (2010),20000000.0,50353641.0,2.517682
8005,The Nun (2018),22000000.0,53807379.0,2.445790
6933,Pitch Perfect 2 (2015),29000000.0,69216890.0,2.386789


In [549]:
movies.columns

Index(['url', 'cast', 'opening_wknd_bo', 'legs', 'dom_pct', 'adj_dom_bo',
       'budget', 'runtime', 'keywords', 'source', 'prod_method',
       'creative_type', 'producers', 'prod_countries', 'languages',
       'franchise', 'dom_bo', 'int_bo', 'ww_bo', 'est_dom_dvd',
       'est_dom_bluray', 'tot_dom_vid_sales', 'year', 'movie_title', 'genre',
       'revenue_to_date', 'prod_multiple', 'opening_wknd_bo_pct_tot',
       'n_opening_theaters', 'n_max_theaters', 'weeks_average_run_theater',
       'movie_id', 'mpaa', 'Director', 'Producer', 'Executive Producer',
       'Editor', 'Composer', 'Screenwriter', 'Production Designer',
       'Costume Designer', 'Story Creator', 'Director of Photography',
       'Based on', 'Casting Director', 'dom_release_date',
       'video_release_date', 'dom_release_date_weekday',
       'dom_release_date_week_um', 'diff_dom_video_release',
       'opening_v_budget'],
      dtype='object')

In [571]:
movies.explode('Director').groupby('Director').sum().reset_index().iloc[0, 0]

/var/folders/f4/mmfbs50954l08xffq4m0vf3m0000gn/T/ipykernel_59219/3483731357.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  movies.explode('Director').groupby('Director').sum().reset_index().iloc[0, 0]


''

In [575]:
movies['Director'][:50]

0                                       [D.W. Griffith]
1                                        [Stuart Paton]
2                                          [Rex Ingram]
3                                          [King Vidor]
4                                          [Fred Niblo]
5                                                      
6                                                      
7                                         [James Whale]
8              [Merian C. Cooper, Ernest B. Schoedsack]
9                                                      
10                                      [W.S. Van Dyke]
11                                         [David Hand]
12    [Mervyn LeRoy, King Vidor, Victor Fleming, Ric...
13                                        [Frank Capra]
14             [Victor Fleming, George Cukor, Sam Wood]
15                                                     
16                                        [Jack Conway]
17                                   [Alfred Hit

In [576]:
def load_movies():
    # Load and preprocess your data here
    movies = pd.read_pickle('processed_data/movies.pkl.gz')
    # Perform any necessary preprocessing
    return movies

In [578]:
test = load_movies()

In [588]:
test['Director'][0]

['D.W. Griffith']

In [591]:
movies['Executive Producer'].apply(eval())

SyntaxError: invalid syntax (<string>, line 0)

In [583]:
test['Executive Producer'][:50]

0                   
1                   
2                   
3                   
4                   
5                   
6                   
7                   
8                   
9                   
10                  
11                  
12                  
13                  
14                  
15                  
16                  
17                  
18                  
19                  
20                  
21                  
22                  
23                  
24                  
25                  
26                  
27                  
28    [B.G. DeSylva]
29                  
30                  
31                  
32                  
33                  
34                  
35                  
36                  
37                  
38                  
39                  
40                  
41                  
42                  
43                  
44                  
45                  
46                  
47           